In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/cambrian/'
data_folder = parent_folder + 'data/'
from hydra import compose, initialize
with initialize(config_path="../../conf"):
    # Compose the configuration
    cfg = compose(config_name="TopicClusterMapping.yaml")

import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_91784/4032682437.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="../../conf"):


In [2]:
import requests

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/cambrian/')
from src.onboarding.TopicClusterMapping import TopicClusterMapping

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-pac

In [3]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [4]:
with PostgresDatabaseOperation() as cursor:
    sql = 'select * from topic_cluster_preference_mapping'
    cursor.execute(sql)
    results = cursor.fetchall()

In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.DataFrame(results, columns = ['id','topic_id', 'topic_name', 'cluster_id', 'a', 'b', 'run_id', 'created_at'])

In [4]:
from src.onboarding.TopicClusterMapping import TopicClusterMapping

In [5]:
tcm = TopicClusterMapping(cfg=cfg)

In [6]:
topics = tcm._get_all_topics()

In [21]:
import openai

In [22]:
openai.api_key = ''

In [142]:
def get_representations(topics, method = 'category: name'):
    representations = []
    if method == 'category: name':
        representations = [{'topicId': x['topicId'], 'description': f"{x['category']}: {x['topicName']}", 'approach_name': method} for x in topics]
        return representations
    elif method == 'gpt3-article':
        for x in tqdm(topics):
            representations.append({'topicId': x['topicId'], 'description': get_gpt3_article_on_topic(category = x['category'], topic = x['topicName']), 'approach_name': method})
        return representations

In [23]:
def get_gpt3_article_on_topic(category, topic):
    prompt = f"""
    I am looking to read information and content related to the topic of {topic} under the category of {category}. Recommend what are the kind of articles and content that i should looking at. what kind of information and concepts would be present in these content. some of these will be news about latest events, some will be opinion pieces and some will be learning modules. structure your answer like you're written an article on the best way to identifying these articles. Your answer should have the following sections:
1. give a brief introduction about the topic and category. be brief about not more than 2-3 lines on this
2. technical aspects and background which i should be aware of to get a good handle on the topic. be extensive here. for example advanced financial concepts would include a wide range from black scholes models, F&Os to how to value a company.  
3. what are some of the common examples of these topics. for example crypocurrency has a wide range from bitcoin, ethereum etc. 
4. then for each of these 3 types of content, mention 10 common article titles that i should expect. I am indian financial content reader, so tailor your recommendations accordingly. make sure introduce diversity in the titles so that they reflect the diversity of articles i would to expect to find

use an article kind of structure with title and body and not a bullet point approach. no preamble. be concise.
    """
    # return prompt
    res = openai.ChatCompletion.create(model = "gpt-3.5-turbo", messages = [{'role': 'system', 'content': ''}, {'role': 'user', 'content': prompt}])
    return res['choices'][0]['message']['content']

In [26]:
def get_gpt4_article_on_topic(category, topic):
    prompt = f"""
    I am looking to read information and content related to the topic of {topic} under the category of {category}. Recommend what are the kind of articles and content that i should looking at. what kind of information and concepts would be present in these content. some of these will be news about latest events, some will be opinion pieces and some will be learning modules. structure your answer like you're written an article on the best way to identifying these articles. Your answer should have the following sections:
1. give a brief introduction about the topic and category. be brief about not more than 2-3 lines on this
2. technical aspects and background which i should be aware of to get a good handle on the topic. be extensive here. for example advanced financial concepts would include a wide range from black scholes models, F&Os to how to value a company.  
3. what are some of the common examples of these topics. for example crypocurrency has a wide range from bitcoin, ethereum etc. 
4. then for each of these 3 types of content, mention 10 common article titles that i should expect. I am indian financial content reader, so tailor your recommendations accordingly. make sure introduce diversity in the titles so that they reflect the diversity of articles i would to expect to find

use an article kind of structure with title and body and not a bullet point approach. no preamble. be concise.
    """
    # return prompt
    res = openai.ChatCompletion.create(model = "gpt-4", messages = [{'role': 'system', 'content': ''}, {'role': 'user', 'content': prompt}])
    return res['choices'][0]['message']['content']

In [48]:
topics[40]

{'topicId': 41,
 'topicName': 'GST & Other Tax Regulations',
 'category': 'Regulatory & Compliance',
 'createdAt': '2023-10-06T06:51:44.276+00:00'}

In [171]:
r = get_gpt3_article_on_topic(topics[29]['category'],topics[29]['topicName'])

In [174]:
r4 = get_gpt4_article_on_topic(topics[29]['category'],topics[29]['topicName'])

In [179]:
r41 = get_gpt4_article_on_topic(topics[29]['category'],topics[29]['topicName'])

In [155]:
from src.embeddings._utils import __clean_article_content

In [60]:
import re

In [75]:
def __clean_article_content(article_content: str, article_id: str):
    # removing new line symbol
    article_content = article_content.replace('\n', ' ')
    article_content = article_content.replace('\t', ' ')
    # removing characters which are not part of alphabets, numbers, currencies, percent, dot, dash, brackets, space, slash
    article_content = re.sub(r"[^a-zA-Z$€₹%\-:/()\[\]{} .]", "", article_content)
    return article_content

In [50]:
from tqdm.notebook import tqdm

In [54]:
representations = []
for x in tqdm(topics):
    representations.append({'topicId': x['topicId'], 'description': get_gpt3_article_on_topic(category = x['category'], topic = x['topicName']), 'approach_name': 'gpt3-article'})

  0%|          | 0/45 [00:00<?, ?it/s]

In [76]:
cleaned_representations = []
for x in representations:
    cleaned_representations.append({'topicId': x['topicId'], 'description': __clean_article_content(x['description'],'1'), 'approach_name': x['approach_name']})

In [128]:
# res = openai.ChatCompletion.create(model = 'gpt-3.5-turbo', messages = [{'role': 'system', 'content': ''}, {'role': 'user', 'content': text}])

In [73]:
def write_representations_to_db(topic_representations):
    with PostgresDatabaseOperation() as cursor:
        insert_sql = """
        INSERT INTO topic_internal_representations (topic_id, description, approach_name)
            VALUES (%s, %s, %s)
            ON CONFLICT (topic_id, approach_name)
            DO UPDATE SET 
                topic_id = EXCLUDED.topic_id,
                description = EXCLUDED.description,
                approach_name = EXCLUDED.approach_name;
        """
        for topic in topic_representations:
            cursor.execute(insert_sql, (topic['topicId'], topic['description'], topic['approach_name']))

In [78]:
write_representations_to_db(topic_representations=cleaned_representations)

### Embeddings

In [82]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.embeddings._utils import parallel_get_article_metadata_from_api

In [41]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT article_id, embedding FROM embeddings'
    cursor.execute(sql)
    results = cursor.fetchall()
    converted_results = [(x[0], [float(y) for y in x[1]]) for x in results]

emb_df = pd.DataFrame(converted_results, columns = ['article_id', 'embedding'])
all_articles = list(emb_df['article_id'].unique())
metadata = parallel_get_article_metadata_from_api(all_articles)
emb_df['title'] = emb_df['article_id'].apply(lambda x: metadata[x]['title'])

In [44]:
import numpy as np

### Closest articles

In [45]:
def get_top_closest_articles(search_term, n = 10):
    emb = tcm.bertopic_model.embedding_model.embed_words(search_term)
    sim_arr = np.array([cosine_similarity(np.array(emb_df.iloc[n]['embedding']).reshape(-1,1).T, np.array(emb).reshape(-1,1).T)[0][0] for n in range(len(emb_df))])
    top_articles = sim_arr.argsort()[-n:][::-1]
    return [emb_df.iloc[i]['title'] for i in top_articles], [sim_arr[i] for i in top_articles]

In [81]:
get_top_closest_articles('Investment: P2P Lending')

(['Peer-to-Peer Lending: Advantages & Disadvantages & How it Works?',
  'P2P Lending 101 – Ultimate Beginner’s Guide',
  'What Happens If a Borrower Defaults on a P2P Loan?',
  'Gold Loan - Everything You Need to Know',
  'Investing vs Loan Repayment - Which One to Prioritise?',
  'Rs. 100 SIPs per Month: Minimum Investment Plans for Mutual Funds',
  'Project lender Ethos aims to deploy USD 1 billion for lending in India over 5 years',
  'RD vs SIP: A Comparative Analysis',
  'Investing in Initial Public Offerings (IPOs) - Good or Bad?￼',
  'Best Investment Options in India in 2023'],
 [0.7743737287368375,
  0.7457288965849806,
  0.7311486087527139,
  0.6439314734592184,
  0.6409045857196916,
  0.6368065217255401,
  0.63232730331419,
  0.6317944443800791,
  0.6294705632068782,
  0.629385694476484])

In [47]:
get_top_closest_articles(r)

(['Peer-to-Peer Lending: Advantages & Disadvantages & How it Works?',
  'P2P Lending 101 – Ultimate Beginner’s Guide',
  'What Happens If a Borrower Defaults on a P2P Loan?',
  'How Can You Avail a Startup Business Loan?',
  'Best Investment Options in India in 2023',
  'What Are Alternate Investment Funds (AIFs)- All You Need Know',
  'How to Choose Mutual Funds in India?',
  'What is Private Equity Investment',
  'RBI’s Public Tech Platform for Frictionless Credit',
  'Mutual Funds Vs Bonds: Learn the key differences'],
 [0.861496868938388,
  0.8333886014398606,
  0.7743011924454906,
  0.7142748989874288,
  0.6939185757146832,
  0.6826027181712353,
  0.6824112232779447,
  0.68053892088434,
  0.6745454970499052,
  0.6733137134997268])

### Similar clusters

In [8]:
with PostgresDatabaseOperation() as cursor:
    cursor.execute('SELECT parent_id, parent_name FROM cluster_hierarchy')
    results = cursor.fetchall()

In [10]:
cluster_names = {}
for tup in results:
    cluster_names[tup[0]] = tup[1]

In [55]:
def format_cluster_priors(priors_dict):
    formatted_priors = {}
    for cluster in priors_dict.keys():
        if priors_dict[cluster]['a'] > 1:
            formatted_priors[cluster_names[cluster]] = round(priors_dict[cluster]['a'], 2)
    return {k: v for k, v in sorted(formatted_priors.items(), key=lambda item: item[1])[::-1]}

In [79]:
format_cluster_priors(tcm._compute_cluster_priors_for_keyword(word = 'Investment: P2P Lending'))

{'bank_rbi_loan_credit_banks': 3.72,
 'crypto_target price_price rs_bitcoin_buy': 2.31,
 'projects_cr_fpis_rs_equities': 1.65,
 'ipo_adani_papers_papers sebi_power': 1.46}

In [80]:
format_cluster_priors(tcm._compute_cluster_priors_for_keyword(word = r))

{'bank_rbi_loan_credit_banks': 4.01,
 'crypto_target price_price rs_bitcoin_buy': 2.2,
 'ipo_adani_papers_papers sebi_power': 1.48,
 'air_flight_airport_air india_india': 1.45}

### Get representations from DB

In [86]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [87]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT * FROM topic_internal_representations'
    cursor.execute(sql)
    results = cursor.fetchall()
representations_df = pd.DataFrame(results, columns = ['id', 'topic_id', 'description', 'approach_name', 'created_at'])

In [96]:
def get_topic_name_and_description(topic_id, approach_name):
    if topic_id not in list(representations_df.topic_id.unique()):
        return None, None
    topic_name = representations_df[(representations_df.topic_id == topic_id) & (representations_df.approach_name == 'category: name')].iloc[0]['description']
    df = representations_df[(representations_df.topic_id == topic_id) & (representations_df.approach_name == approach_name)]
    if df.empty:
        return topic_name, None
    else:
        description = df.iloc[0]['description']
        return topic_name, description

In [104]:
def get_top_articles_for_topic(topic_id, approach_name, n = 20):
    topic_name, description = get_topic_name_and_description(topic_id=topic_id, approach_name=approach_name)
    if description and topic_name:
        articles, sim = get_top_closest_articles(search_term=description, n = n)
        return topic_name, articles, sim
    else:
        return None

In [101]:
def get_cluster_priors_for_topic(topic_id, approach_name):
    topic_name, description = get_topic_name_and_description(topic_id=topic_id, approach_name=approach_name)
    if description and topic_name:
        priors = format_cluster_priors(tcm._compute_cluster_priors_for_keyword(word = description))
        return priors

In [124]:
from tqdm.notebook import tqdm

In [129]:
lst = []
for i in range(1,10):
    dct = {}
    dct['a'] = i
    dct['b'] = 1/i
    lst.append(dct)

In [131]:
topic_maps = []
for topic_id in tqdm(range(1,46)):
    cur_topic_dict = {}
    cur_topic_dict['topic_id'] = topic_id
    for approach in ['gpt3-article', 'category: name']:
        cur_topic_dict[f'cluster_priors_{approach}'] = get_cluster_priors_for_topic(topic_id=topic_id, approach_name=approach)
        _, cur_topic_dict[f'top_articles_{approach}'], _  = get_top_articles_for_topic(topic_id = topic_id, approach_name=approach)
    topic_maps.append(cur_topic_dict)

  0%|          | 0/45 [00:00<?, ?it/s]

In [134]:
topic_comparison_df = pd.DataFrame(topic_maps)

In [183]:
topic_comparison_df['random_number'] = [np.random.randint(1000, 2000) for i in range(len(topic_comparison_df))]

In [186]:
topic_comparison_df['number_gpt3-article'] = topic_comparison_df['random_number'].apply(lambda x: prime(x * 2))
topic_comparison_df['number_category: name'] = topic_comparison_df['random_number'].apply(lambda x: prime(x * 2 + 1))

In [187]:
topic_comparison_df

,topic_id,cluster_priors_gpt3-article,top_articles_gpt3-article,cluster_priors_category: name,top_articles_category: name,random_number,number_gpt3-article,number_category: name
0,1,"{'bank_rbi_loan_credit_banks': 3.39, 'crypto_t...","[Best Investment Options in India in 2023, 10 ...","{'bank_rbi_loan_credit_banks': 2.67, 'inflatio...",[10 Smart Money-Saving Tips to achieve your fi...,1823,34127,34129
1,2,"{'bank_rbi_loan_credit_banks': 4.03, 'air_flig...","[How to Handle Credit Card Debt Responsibly?, ...","{'bank_rbi_loan_credit_banks': 3.42, 'ipo_adan...","[How to Handle Credit Card Debt Responsibly?, ...",1489,27191,27197
2,3,"{'pension_epfo_higher_eps_eps pension': 2.22, ...",[Features & Benefits of Pension Plans in India...,"{'pension_epfo_higher_eps_eps pension': 2.16, ...",[Step-By-Step Guide to Efficient Retirement Pl...,1338,24061,24071
3,4,"{'bank_rbi_loan_credit_banks': 2.43, 'gst_tax_...","[6 Tax Planning Strategies for New Businesses,...","{'gst_tax_pli_gst council_dpiit': 1.95, 'pensi...","[6 Tax Planning Strategies for New Businesses,...",1324,23813,23819
4,5,"{'irdai_life_insurance_insurers_lic': 2.37, 'd...","[All You Need to Know about Term Insurance , T...","{'irdai_life_insurance_insurers_lic': 2.24, 'b...",[Things to Check before You Buy Life Insurance...,1078,18959,18973
5,6,"{'bank_rbi_loan_credit_banks': 3.1, 'irdai_lif...",[Details of Rental Income Taxation in India (2...,"{'bank_rbi_loan_credit_banks': 2.6, 'projects_...",[Real Estate vs Mutual Funds - Which Is the Be...,1830,34261,34267
6,7,"{'stocks_52 week_52_week_nifty': 3.13, 'crypto...","[Stock Market Indices: Meaning, Types, and Oth...","{'stocks_52 week_52_week_nifty': 2.83, 'crypto...",[Beginners Stock Market Guide - Start Investin...,1096,19373,19379
7,8,"{'stocks_52 week_52_week_nifty': 2.97, 'ipo_ad...",[Beginners Stock Market Guide - Start Investin...,"{'stocks_52 week_52_week_nifty': 2.86, 'projec...","[Why You Should Choose Equity Over FDs, Gold &...",1020,17783,17789
8,9,"{'stocks_52 week_52_week_nifty': 2.88, 'bank_r...","[How to Choose Mutual Funds in India?, Mutual ...","{'stocks_52 week_52_week_nifty': 2.66, 'crypto...",[How to Invest in Mutual Funds Online - A Comp...,1168,20747,20749
9,10,{'crypto_target price_price rs_bitcoin_buy': 2...,[Index Fund vs ETFs: What Are the Differences?...,{'crypto_target price_price rs_bitcoin_buy': 2...,"[All You Need to Know about Silver ETFs, Mutua...",1534,28111,28123


In [207]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [209]:
with PostgresDatabaseOperation() as cursor:
    sql = """SELECT source_name, COUNT(is_cleaned_completely) total, SUM(CAST(is_cleaned_completely AS INT)) clean 
    FROM article_qc_table
    GROUP BY source_name
    """
    cursor.execute(sql)
    results = cursor.fetchall()
    source_count_df = pd.DataFrame(results, columns = ['source_name', 'total', 'clean'])

In [210]:
source_count_df

,source_name,total,clean
0,economic-times,4,4
1,wint-wealth,1,0
2,live-mint,3,0


In [217]:
source_count_df[['source_name','total']].set_index('source_name').to_dict()['total']

{'economic-times': 4, 'wint-wealth': 1, 'live-mint': 3}

In [156]:
!pip install sympy

In [162]:
from sympy import primepi, isprime

def find_prime_position(number):
    # First, check if the number is prime, as primepi function counts the primes less than or equal to the number
    # and doesn't directly give the position of a specific prime.
    if isprime(number):
        # If the number is prime, find its position among primes.
        position = primepi(number)
        return position
    else:
        return None  # or raise an exception, or any other way you'd like to handle non-prime numbers

# Test with a prime number
prime_number = 29
position = find_prime_position(prime_number)

if position:
    print(f"The number {prime_number} is the {position}th prime.")
else:
    print(f"The number {prime_number} is not a prime.")

The number 29 is the 10th prime.


In [157]:
from sympy import prime

In [200]:
def format_text(x):
    x = x.replace("{",'')
    x = x.replace("}",'')
    x = x.replace("[",'')
    x = x.replace("]",'')
    x = x.replace("'",'')
    x = x.replace(':', ' -> ')
    x = x.replace(',', ' \n ')
    return x

In [197]:
format_text(str(topic_comparison_df.iloc[0]['cluster_priors_gpt3-article']))

In [201]:
topic_comparison_df['cluster_priors_gpt3-article'].apply(lambda x: format_text(str(x)))

0     bank_rbi_loan_credit_banks ->  3.39 \n  crypto...
1     bank_rbi_loan_credit_banks ->  4.03 \n  air_fl...
2     pension_epfo_higher_eps_eps pension ->  2.22 \...
3     bank_rbi_loan_credit_banks ->  2.43 \n  gst_ta...
4     irdai_life_insurance_insurers_lic ->  2.37 \n ...
5     bank_rbi_loan_credit_banks ->  3.1 \n  irdai_l...
6     stocks_52 week_52_week_nifty ->  3.13 \n  cryp...
7     stocks_52 week_52_week_nifty ->  2.97 \n  ipo_...
8     stocks_52 week_52_week_nifty ->  2.88 \n  bank...
9     crypto_target price_price rs_bitcoin_buy ->  2...
10    projects_cr_fpis_rs_equities ->  2.16 \n  ipo_...
11    bank_rbi_loan_credit_banks ->  3.34 \n  ipo_ad...
12    stocks_52 week_52_week_nifty ->  2.54 \n  cryp...
13    crypto_target price_price rs_bitcoin_buy ->  2...
14    bank_rbi_loan_credit_banks ->  4.2 \n  crypto_...
15    bank_rbi_loan_credit_banks ->  4.65 \n  crypto...
16    bank_rbi_loan_credit_banks ->  5.0 \n  air_fli...
17    bank_rbi_loan_credit_banks ->  5.0 \n  air

In [189]:
import requests
import random

# Set up your headers with your Typeform API key
headers = {
    'Authorization': 'Bearer ',
    'Content-Type': 'application/json',
}

base_url = 'https://api.typeform.com/forms'

def create_form(df, num_sample = 10):
    form_data = {
        'title': 'Survey',
        'type': 'quiz',
        'fields': [],
        'show_progress_bar': True,
        'show_question_number': True
    }

    # Construct fields based on dataframe rows
    for _, row in df.sample(num_sample).iterrows():
        # Prepare choices, shuffling to ensure randomness
        choices_labels = [row['Cluster1'], row['Cluster2']]
        random.shuffle(choices_labels)  # Randomize or shuffle the choices
        choices = [{'label': label} for label in choices_labels]

        # Building the question field
        field = {
            'type': 'multiple_choice',
            'title': 'some title',
            'properties': {'description': row['Topic'], 'choices': choices, 'allow_multiple_selection': False, 'vertical_alignment': False, 'button_text': 'Next'}
        }

        # Append the constructed field to the form fields
        form_data['fields'].append(field)


    # Post the form data to Typeform's API to create a new form
    response = requests.post(base_url, headers=headers, json=form_data)

    if response.status_code == 201:  # A '201 Created' status indicates that the form was successfully created.
        print("Form created successfully!")

        # The response body (JSON) from Typeform once the form is created contains the URL among other data.
        form_response = response.json()

        # Extract the URL of the newly created form. Usually, it can be found under the '_links' section in the key named 'display'.
        form_url = form_response.get('_links', {}).get('display', '')

        print(f"Your new form is live at: {form_url}")

        return form_url  # You might want to return this URL for use in other parts of your application.
    else:
        print(f"Failed to create form. Status code: {response.status_code}, Message: {response.text}")
        return None 

In [120]:
get_cluster_priors_for_topic(topic_id=8, approach_name='gpt3-article')

{'stocks_52 week_52_week_nifty': 2.97,
 'ipo_adani_papers_papers sebi_power': 2.2,
 'crypto_target price_price rs_bitcoin_buy': 1.76,
 'sebi_case_case sebi_entities_disclosure': 1.72,
 'projects_cr_fpis_rs_equities': 1.54,
 'air_flight_airport_air india_india': 1.47}

{'stocks_52 week_52_week_nifty': 2.86,
 'projects_cr_fpis_rs_equities': 2.15,
 'ipo_adani_papers_papers sebi_power': 2.13,
 'crypto_target price_price rs_bitcoin_buy': 1.71,
 'inflation_growth_rate_fed_fy24': 1.56}

In [126]:
_, top_articles, _  = get_top_articles_for_topic(topic_id = 8, approach_name='gpt3-article')

In [123]:
get_top_articles_for_topic(topic_id = 8, approach_name='category: name')

('Investment: Equities',
 ['Why You Should Choose Equity Over FDs, Gold & Real Estate',
  'Mutual funds invest Rs 1.82 lakh cr in equities in FY23 on strong push from retail investors',
  "A Beginner's Guide to Equity Investment",
  'Retail investors’ ever-lasting love for equity mutual funds continues to blossom | Mint',
  'Retail investors’ ever-lasting love for equity mutual funds continues to blossom | Mint',
  'Better to play real estate through related sectors: KR Senthilnathan, NAFA',
  '5 Rules of Equity Investment',
  'Best Investment Options in India in 2023',
  'Investing in mutual funds in India: Why 2023 is the year to take action',
  'How to Invest in Mutual Funds Online - A Complete Guide',
  'FPIs buying spree continues; invest Rs 9,800 cr in equities this month',
  '7 Debt Investment Options Which Give Decent Returns in India - 2023',
  'Long Term Capital Gain on Mutual Funds - Tax Exemptions and Calculations',
  'Short-Term vs Long-Term Investments - Which Is Better?'

In [117]:
from src.embeddings._utils import get_article_metadata_from_api
def get_metadata_for_title(title):
    df = emb_df[emb_df.title == title]
    if not df.empty:
        article_id = df.iloc[0].article_id
        return get_article_metadata_from_api(article_id)

In [88]:
representations_df

,id,topic_id,description,approach_name,created_at
0,1,1,Personal Finance: Budgeting & Saving,category: name,2023-10-19 06:44:48.089906+00:00
1,2,2,Personal Finance: Debt Management,category: name,2023-10-19 06:44:48.089906+00:00
2,3,3,Personal Finance: Retirement Planning,category: name,2023-10-19 06:44:48.089906+00:00
3,4,4,Personal Finance: Tax Planning,category: name,2023-10-19 06:44:48.089906+00:00
4,5,5,"Personal Finance: Insurance (Life, Health, Aut...",category: name,2023-10-19 06:44:48.089906+00:00
...,...,...,...,...,...
85,92,39,. Introduction: RBI regulations refer to the g...,gpt3-article,2023-10-19 10:50:42.207429+00:00
86,93,40,. Introduction: Financial Audits Reporting is...,gpt3-article,2023-10-19 10:50:42.207429+00:00
87,95,42,. Introduction Agricultural Finance is a spec...,gpt3-article,2023-10-19 10:50:42.207429+00:00
88,97,44,. Introduction: Sustainable Green Finance is ...,gpt3-article,2023-10-19 10:50:42.207429+00:00
